In [45]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import warnings 
warnings.filterwarnings('ignore')

In [46]:
df = pd.read_csv('train.csv')
df = pd.get_dummies(df)

In [47]:
text_to_find = 'noninvasive'
columns_to_drop = [col for col in df.columns if text_to_find in col]
df.drop(columns=columns_to_drop, inplace=True)

In [48]:
from sklearn.impute import KNNImputer

imr = KNNImputer(n_neighbors=10001, weights='distance')
imr = imr.fit(df.values)
df[:]= imr.transform(df.values)


In [49]:
X = df.loc[:, df.columns != 'hospital_death']
y = df[['hospital_death']]

In [50]:
numeric_columns = df.select_dtypes(include=['int64','float64']).columns

In [51]:
numeric_columns = numeric_columns.drop(['RecordID', 'hospital_id', 'icu_id', 'hospital_death'])

In [52]:
from sklearn.preprocessing import RobustScaler

rbs = RobustScaler()
df[numeric_columns] = rbs.fit_transform(df[numeric_columns])
df

,RecordID,hospital_id,icu_id,age,elective_surgery,pre_icu_los_days,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,gcs_eyes_apache,...,apache_2_bodysystem_Cardiovascular,apache_2_bodysystem_Gastrointestinal,apache_2_bodysystem_Haematologic,apache_2_bodysystem_Metabolic,apache_2_bodysystem_Neurologic,apache_2_bodysystem_Renal/Genitourinary,apache_2_bodysystem_Respiratory,apache_2_bodysystem_Trauma,apache_2_bodysystem_Undefined Diagnoses,apache_2_bodysystem_Undefined diagnoses
0,1,126,1931,-1.636364,0.0,-0.366,0.000000,0.592016,0.0,-1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,112,1544,0.227273,0.0,-0.284,0.957447,-0.595816,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,153,1517,0.909091,0.0,31.780,-0.085106,-0.389984,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,4,109,1811,-0.227273,0.0,-0.132,0.952128,-0.008000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,5,287,1845,0.954545,0.0,-0.302,-0.026596,-0.601976,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,49996,110,1689,-1.000000,0.0,-0.394,0.989362,1.389944,0.0,-1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
49996,49997,155,1719,-0.136364,0.0,13.500,-0.010638,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
49997,49998,93,1912,-0.636364,0.0,-0.392,-0.047872,0.188072,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
49998,49999,230,1720,-0.454545,1.0,0.816,0.425532,1.607936,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
X = df.loc[:, df.columns != 'hospital_death']
y = df[['hospital_death']]

In [54]:
df_test = pd.read_csv('test.csv')
df_test = pd.get_dummies(df_test)


columns_to_drop = [col for col in df_test.columns if text_to_find in col]
df_test.drop(columns=columns_to_drop, inplace=True)

imr_test = KNNImputer(n_neighbors=10001, weights='distance')
imr_test = imr_test.fit(df_test.values)
df_test[:]= imr_test.transform(df_test.values)

rbs_test = RobustScaler()
df_test[numeric_columns] = rbs_test.fit_transform(df_test[numeric_columns])

In [55]:
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import csv

scale_knn = Pipeline([("scaler", RobustScaler()), ("knr", KNeighborsClassifier(n_neighbors=1269,weights='distance',p=1))])
scale_knn.fit(X,y)
md_probs = scale_knn.predict_proba(df_test)
md_probs = md_probs[:,1]
filepath = 'predictknn.csv'
with open(filepath, mode='w', newline='') as file: 
    writer = csv.writer(file)
    c=1
    for i in md_probs:
        writer.writerow([50000+c , i])
        c=c+1